In [ ]:
%%javascript
document.title='fuzzy-lasagna'

In [ ]:
import os
import kitchen.ingredients as k
import dropkick as dk
import sys; sys.path.append("../../../../utility_scripts/")
from mxif_immexcl_analysis import *

# some stuff to make this notebook work better with Scanpy
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=400)
sc.settings.figdir = "segmentation/"

In [ ]:
sns.set_style("white")

In [ ]:
sample_key = pd.read_csv("../../../resources/visium_sample_key.csv", index_col=0)

sample_key.ws_adata = sample_key.ws_adata.fillna("")

In [ ]:
celltype_dict = {
    'T helper': ["CD3D","CD4"],
    'T reg': ["CD3D","CD4","FOXP3"],
    'T cytotoxic': ["CD3D","CD8"],
    'Myeloid': ["CD11B"],
    'Macrophage': ["CD11B","CD68"],
    'Macrophage M1': ["CD11B","CD68","LYSOZYME"],
}

---
# Read outputs from `07-MxIF_immexcl_analysis.ipynb`

In [ ]:
out = pd.DataFrame()
for file in os.listdir("segmentation/"):
    if "immunecell_cnvclone_quant.csv" in file:
        print(file)
        block_name = file.split("_")[0]
        pat_name = sample_key.loc[sample_key.block_name==block_name, "patient_name"][0]
        a = pd.read_csv("segmentation/{}".format(file), index_col=0)
        a.index = [pat_name + " " + x for x in a.index]
        out = pd.concat([out, a])

In [ ]:
out

In [ ]:
out.plot.barh(figsize=(5,15))
plt.legend(bbox_to_anchor=(1, 1), loc="upper left", fontsize="small", frameon=False)
sns.despine()
#plt.tight_layout()
#plt.savefig("segmentation/PAT71662_PAT73458_immunecell_cnvclone_quant.png")

In [ ]:
out.loc[(out.index.str.startswith("PAT71662"))|(out.index.str.startswith("PAT73458")),:].plot.barh(figsize=(5,8))
plt.legend(bbox_to_anchor=(1, 1), loc="upper left", fontsize="small", frameon=False, title="Cell Type (MxIF)")
plt.xlabel("# of Cells (MxIF)")
sns.despine()
plt.tight_layout()
plt.savefig("segmentation/PAT71662_PAT73458_immunecell_cnvclone_quant.png")

In [ ]:
out.loc[(out.index.str.startswith("PAT71662"))|(out.index.str.startswith("PAT73458")),["T cytotoxic","T helper"]].plot.bar(figsize=(8,4), color={"T cytotoxic":"red","T helper":"purple"})
plt.legend(bbox_to_anchor=(1, 1), loc="upper left", fontsize="small", frameon=False, title="Cell Type (MxIF)")
plt.ylabel("# of Cells (MxIF)")
sns.despine()
plt.tight_layout()
plt.savefig("segmentation/PAT71662_PAT73458_lymphocyte_cnvclone_quant.png")

---

In [ ]:
meta_df = pd.read_csv("../../visium/CNVclone_heatmap_df.csv")

In [ ]:
final = out.merge(meta_df[["CNV Clone","CIN+ PPT (CNV Score)","HM PPT (Mut. Burden)","Tumor Type","Immune Exclusion","CIN Status"]], left_index=True, right_on="CNV Clone", how="left").dropna().reset_index(drop=True)

In [ ]:
final = final.melt(id_vars=["CNV Clone","Tumor Type","CIN+ PPT (CNV Score)","HM PPT (Mut. Burden)", "Immune Exclusion","CIN Status"])

In [ ]:
final.rename(columns={"variable":"Cell Type","value":"# of Cells (MxIF)", "CIN Status":"Tumor class","Immune Exclusion":"IES"}, inplace=True)

In [ ]:
final["CIN_PPT_norm"] = final["CIN+ PPT (CNV Score)"] - final["CIN+ PPT (CNV Score)"].min()
final["CIN_PPT_norm"] = final["CIN_PPT_norm"] / final["CIN_PPT_norm"].max()

In [ ]:
final["HM_PPT_norm"] = final["HM PPT (Mut. Burden)"] - final["HM PPT (Mut. Burden)"].min()
final["HM_PPT_norm"] = final["HM_PPT_norm"] / final["HM_PPT_norm"].max()

In [ ]:
final["PPT"] = final["CIN_PPT_norm"]
final.loc[final["Tumor class"]=="HM", "PPT"] = final.loc[final["Tumor class"]=="HM", "HM_PPT_norm"]

In [ ]:
cmap_dict = {
    "SSL/HP":"#c4a4e1","MSI-H":"#7a4fa3","MSS":"#ffc101","TA/TVA":"#fee799","NL":"#1f77b4",
    "CIN+":"#ffc101","HM":"#7a4fa3",
    "Cecum":"#1f4e79","Ascending":"#2e74b7","Hepatic Flexure":"#bdd6ef","Transverse":"#ff717a","Descending":"#fe0001","Sigmoid":"#c00101",
    "lymph_node":"#000000",
    "nan":"#ffffff",
    "T":"#000000","F":"#ffffff",
    "N":"#1f77b4","L":"#ff7f0e","B":"#2ca02c",
    "DOD":"#ff0000","DOC":"#ff0000","AWD":"#ffff00","NED":"#008000",
}

In [ ]:
plt.figure(figsize=(5.5, 3.5))
#sns.regplot(data=final, x="Immune Exclusion", y="# of Cells (MxIF)", scatter=False, line_kws={"alpha":0.3, "color":"k"})#, order=3)
sns.scatterplot(data=final, x="IES", y="# of Cells (MxIF)", hue="Tumor class", size="PPT", palette=cmap_dict)
plt.legend(bbox_to_anchor=(1, 1), loc="upper left", fontsize="small", frameon=False)
sns.despine()
plt.tight_layout()
plt.savefig("segmentation/immunecell_cnvclone_quant.png", dpi=400)

In [ ]:
g = sns.FacetGrid(final, col="Cell Type", col_order= ["T reg","Myeloid","T helper","Macrophage","T cytotoxic","Macrophage M1"], sharey=False, col_wrap=2, aspect=1.2, height=3.5)
g.map(sns.regplot, "IES", "# of Cells (MxIF)", scatter=False, line_kws={"alpha":0.3, "color":"k"})
g.map(sns.scatterplot, "IES", "# of Cells (MxIF)", "Tumor class", "PPT", alpha=0.7, palette=cmap_dict)
g.set_titles(col_template="{col_name}")
g.tight_layout()
g.savefig("segmentation/immunecell_cnvclone_quant_facet.png", dpi=400)

---
# Cell Centroid Viz. - per Sample

---

In [ ]:
pat = "PAT71662"

In [ ]:
a = sc.read("segmentation/WD87480_immunecells.h5ad")
a = a[a.obs.epi==0,:].copy(); a

In [ ]:
a.obsm["spatial_ws"] = flip_coords(a.obsm["spatial_ws"])

In [ ]:
a.obs["Myeloid"] = np.nan
a.obs.loc[a.obs["Cell Type"].isin(["Macrophage","Macrophage M1","Myeloid"]), "Myeloid"] = a.obs.loc[a.obs["Cell Type"].isin(["Macrophage","Macrophage M1","Myeloid"]), "Cell Type"]
a.obs["Lymphoid"] = np.nan
a.obs.loc[a.obs["Cell Type"].isin(["T cytotoxic","T helper","T reg"]), "Lymphoid"] = a.obs.loc[a.obs["Cell Type"].isin(["T cytotoxic","T helper","T reg"]), "Cell Type"]

In [ ]:
cdict = dict(zip(list(a.obs["Cell Type"].cat.categories), list(a.uns["Cell Type_colors"])))
a.uns["Myeloid_colors"] = [cdict[x] for x in a.obs["Myeloid"].cat.categories]
a.uns["Lymphoid_colors"] = [cdict[x] for x in a.obs["Lymphoid"].cat.categories]

In [ ]:
sc.pl.spatial(
    a,
    basis="spatial_ws",
    color=["Lymphoid", "Myeloid"],
    ncols=4,
    img=None,
    spot_size=50,
    scale_factor=1,
    frameon=False,
    cmap="viridis",
    save="_{}_celltypeIDs_split_immune_mxif_epi0.png".format(pat),\
    na_in_legend=False,
)

---

In [ ]:
pat = "PAT73458"

In [ ]:
a = sc.read("segmentation/WD85806_immunecells.h5ad")
a = a[a.obs.epi==0,:].copy(); a

In [ ]:
a.obsm["spatial_ws"] = flip_coords(a.obsm["spatial_ws"])

In [ ]:
a.obs["Myeloid"] = np.nan
a.obs.loc[a.obs["Cell Type"].isin(["Macrophage","Macrophage M1","Myeloid"]), "Myeloid"] = a.obs.loc[a.obs["Cell Type"].isin(["Macrophage","Macrophage M1","Myeloid"]), "Cell Type"]
a.obs["Lymphoid"] = np.nan
a.obs.loc[a.obs["Cell Type"].isin(["T cytotoxic","T helper","T reg"]), "Lymphoid"] = a.obs.loc[a.obs["Cell Type"].isin(["T cytotoxic","T helper","T reg"]), "Cell Type"]

In [ ]:
a.obs["Cell Type"] = a.obs["Cell Type"].astype("category")

In [ ]:
cdict = dict(zip(list(a.obs["Cell Type"].cat.categories), list(a.uns["Cell Type_colors"])))
a.uns["Myeloid_colors"] = [cdict[x] for x in a.obs["Myeloid"].cat.categories]
a.uns["Lymphoid_colors"] = [cdict[x] for x in a.obs["Lymphoid"].cat.categories]

In [ ]:
sc.pl.spatial(
    a,
    basis="spatial_ws",
    color=["Lymphoid", "Myeloid"],
    ncols=4,
    img=None,
    spot_size=50,
    scale_factor=1,
    frameon=False,
    cmap="viridis",
    save="_{}_celltypeIDs_split_immune_mxif_epi0.png".format(pat),\
    na_in_legend=False,
)